## Running hhalign from the HHsuite
Goal: create hhm files and run hhalign between them for all the relevant domains.

First, clone the HHsuite github for HHalign version 3.0.3 (14-07-2017) from: https://github.com/soedinglab/hh-suite according to their README

Than, install (make + sudo make install) into: /usr/local/hhsuite

After installation, create inside the cloned github directory "databases" directory and download "pfamA_31.0.tgz" from: http://wwwuser.gwdg.de/~compbiol/data/hhsuite/databases/hhsuite_dbs/ (and unzip there). This is the database of hhm (not hmm!) files for pfam-v31, which is the prefered format for using HHsuite tools.

In [1]:
import subprocess
import pandas as pd
import pickle
import os.path
from collections import defaultdict

In [2]:
curr_dir = !pwd
pfam_version = "31"

sim_pairs_df_filtered = pd.read_csv("processed_domains_pairs_for_hhalign.csv", sep=',', index_col=0)
with open(curr_dir[0]+"/../2.parse_Pfam/v"+pfam_version+"/domain_to_pfam_acc_dict.pik", 'rb') as handle:
    domain_to_pfam_acc_dict = pickle.load(handle)    
    
hhsuite_hhm_database_path = "../../../HHsuite/databases/pfam-v"+pfam_version+"/"
hhsuite_hhm_database_filename = "pfam_hhm.ffdata"
hhsuite_hhm_database_idx = "pfam_hhm.ffindex"

In [3]:
def create_domain_hhm_file(domain_pfam_id, start_pos, length):
    
    outfilename = "../10.Prediction/domains_similarity/pfam-v"+pfam_version+"/hhm_files/"+domain_pfam_id+".hhm"
    creating_hhm_cmd = "dd if=../../../HHsuite/databases/pfam-v31/pfam_hhm.ffdata of="+outfilename+" bs=1 count="+length+" skip="+start_pos
    not_important_prints = !$creating_hhm_cmd

In [4]:
def process_query(query, res_dict, direction):
    
    
    for line in query:
        
        #Get the probability of the query and the template to be homologus
        if (line.find("Probab=") >= 0):
            end_of_line = line[line.find("Probab=")+7:]
            prob = end_of_line[:end_of_line.find(" ")]
            res_dict["prob"+str(direction)].append(prob)
        
        #Get the alignment p-value
        if (line.find("E-value=") >= 0):
            end_of_line = line[line.find("E-value=")+8:]
            pval = end_of_line[:end_of_line.find(" ")]
            res_dict["pval"+str(direction)].append(pval)
            
        #Get the alignment raw score
        if (line.find("Score=") >= 0):
            end_of_line = line[line.find("Score=")+6:]
            score = end_of_line[:end_of_line.find(" ")]
            res_dict["score"+str(direction)].append(score)
        
        #Get the number of aligned columns
        if (line.find("Aligned_cols=") >= 0):
            end_of_line = line[line.find("Aligned_cols=")+13:]
            aligned = end_of_line[:end_of_line.find(" ")]
            res_dict["aligned_cols"+str(direction)].append(aligned)
        
        #Get the percentage of aligned residue pairs of the query and the template master sequences that are identical.
        if (line.find("Identities=") >= 0):
            end_of_line = line[line.find("Identities=")+11:]
            ident = end_of_line[:end_of_line.find("%")]
            res_dict["ident_perc"+str(direction)].append(ident)

In [5]:
%%time
res_dict = defaultdict(list)

header = "/usr/local/hhsuite/bin/hhalign -i "
ext = ".hhm"
flag = " -t "
hhm_path = "../10.Prediction/domains_similarity/pfam-v"+pfam_version+"/hhm_files/"
flags = " -hide_cons -hide_pred -hide_dssp -o hhalign_out"

for index, pair in sim_pairs_df_filtered.iterrows():
    #Get the domains names
    dom1 =  pair["sim_dom1"]
    dom2 =  pair["sim_dom2"]
    
    #Get the domains PFAM ids
    dom1_pfam_id = domain_to_pfam_acc_dict[dom1]
    dom2_pfam_id = domain_to_pfam_acc_dict[dom2]
    
    outfilename1 = "../10.Prediction/domains_similarity/pfam-v"+pfam_version+"/hhm_files/"+dom1_pfam_id+".hhm"
    if (os.path.isfile(outfilename1) == False):
        #Get domain 1 indices for fast access
        dom1_idx_cmd = "less "+hhsuite_hhm_database_path+hhsuite_hhm_database_idx+" | grep "+dom1_pfam_id
        dom1_idx_line = !$dom1_idx_cmd
        dom1_idx_line = dom1_idx_line[0].split("\t")
        dom1_start_pos = dom1_idx_line[1]
        dom1_len = dom1_idx_line[2]
        #Create the .hhm (special HHsuite format for HMM profiles) for domain 1
        create_domain_hhm_file(dom1_pfam_id,dom1_start_pos,dom1_len)
    
    outfilename2 = "../10.Prediction/domains_similarity/pfam-v"+pfam_version+"/hhm_files/"+dom2_pfam_id+".hhm"
    if (os.path.isfile(outfilename2) == False):
        #Get domain 2 indices for fast access
        dom2_idx_cmd = "less "+hhsuite_hhm_database_path+hhsuite_hhm_database_idx+" | grep "+dom2_pfam_id
        dom2_idx_line = !$dom2_idx_cmd
        dom2_idx_line = dom2_idx_line[0].split("\t")
        dom2_start_pos = dom2_idx_line[1]
        dom2_len = dom2_idx_line[2]
        #Create the .hhm (special HHsuite format for HMM profiles) for domain 2
        create_domain_hhm_file(dom2_pfam_id,dom2_start_pos,dom2_len)
    
    #Running hhalign direction #1
    script = header+hhm_path+dom1_pfam_id+ext+flag+hhm_path+dom2_pfam_id+ext+flags
    query = !$script
    process_query(query, res_dict, 1)
    #Running hhalign direction #2
    script = header+hhm_path+dom2_pfam_id+ext+flag+hhm_path+dom1_pfam_id+ext+flags
    query = !$script
    process_query(query, res_dict, 2)
    
    if (index % 100 == 0):
        print index

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [6]:
#Add infomration to the domains pairs table
sim_pairs_df_filtered["prob1"] = res_dict["prob1"]
sim_pairs_df_filtered["pval1"] = res_dict["pval1"]
sim_pairs_df_filtered["score1"] = res_dict["score1"]
sim_pairs_df_filtered["aligned_cols1"] = res_dict["aligned_cols1"]
sim_pairs_df_filtered["ident_perc1"] = res_dict["ident_perc1"]

sim_pairs_df_filtered["prob2"] = res_dict["prob2"]
sim_pairs_df_filtered["pval2"] = res_dict["pval2"]
sim_pairs_df_filtered["score2"] = res_dict["score2"]
sim_pairs_df_filtered["aligned_cols2"] = res_dict["aligned_cols2"]
sim_pairs_df_filtered["ident_perc2"] = res_dict["ident_perc2"]

#Save the file with all the results
sim_pairs_df_filtered.to_csv("processed_domains_hhlign_scores.csv", sep='\t')

In [7]:
sim_pairs_df_filtered

,sim_dom1,sim_dom2,prob1,pval1,score1,aligned_cols1,ident_perc1,prob2,pval2,score2,aligned_cols2,ident_perc2
0,IDO,Trp_dioxygenase,99.81,2.5e-26,198.33,109,17,99.79,1.7e-25,193.44,109,17
1,F5_F8_type_C,Glyco_hydro_2_N,0.05,0.81,5.40,1,0,0.06,0.78,4.67,1,0
2,Laminin_N,Glyco_hydro_2_N,0.05,0.82,6.83,1,0,0.04,0.92,6.52,1,0
3,Glyco_hydro_2_N,Ephrin_lbd,0.15,0.47,9.29,1,0,0.16,0.44,9.22,1,0
4,START,Polyketide_cyc,95.78,1.4e-06,33.87,42,5,95.81,9.8e-07,32.61,42,5
5,START,IP_trans,0.09,0.66,8.87,7,14,0.56,0.22,12.11,31,6
6,Polyketide_cyc,IP_trans,0.49,0.21,10.98,1,0,0.38,0.24,10.37,1,0
7,UQ_con,UEV,96.62,1.4e-07,38.89,59,15,96.07,4.4e-07,40.32,59,15
8,2OG-FeII_Oxy_2,FTO_NTD,99.50,1.7e-20,131.87,80,15,99.44,7.6e-20,135.17,80,15
9,2OG-FeII_Oxy_2,2OG-FeII_Oxy_3,63.19,0.0012,22.16,17,24,82.25,0.00024,23.85,17,24
